## [Mestrado] 1.1. Experimentos usando instâncias simuladas (autoencoder) com tsfresh

## Bibliotecas e configurações

In [1]:
# Artifício para alcular tempo total do notebook Jupyter
from datetime import datetime 
start_time = datetime.now()

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import logging
import warnings
import sys
import tensorflow as tf
tf.autograph.set_verbosity(0)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
from math import ceil
from matplotlib import pyplot as plt
from time import time
from pathlib import Path
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import ParameterGrid
from pyod.models.auto_encoder import AutoEncoder

In [3]:
logging.getLogger('tsfresh').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore')

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [5]:
data_path = Path('./', 'data')
random_state = 1
np.random.seed(random_state)
events_names = {0: 'Normal',
                1: 'Aumento Abrupto de BSW',
                2: 'Fechamento Espúrio de DHSV',
                3: 'Intermitência Severa',
                4: 'Instabilidade de Fluxo',
                5: 'Perda Rápida de Produtividade',
                6: 'Restrição Rápida em CKP',
                7: 'Incrustação em CKP',
                8: 'Hidrato em Linha de Produção'
               }
vars = ['P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        'P-JUS-CKGL',
        'T-JUS-CKGL',
        'QGL']
columns = ['timestamp'] + vars + ['class'] 
normal_class_code = 0
abnormal_classes_codes = [1, 2, 5, 6, 7, 8]
sample_size = 3*60              # Nas observações = segundos
min_normal_period_size = 20*60  # Nas observações = segundos
split_range = 0.6               # Porcentagem de separação entre treino/teste
max_samples_per_period = 15     # limitação por 'segurança'
df_fc_p = MinimalFCParameters() # Ver documentação da biblioteca tsfresh
df_fc_p.pop('sum_values')       # Remove feature inapropriada
df_fc_p.pop('length')           # Remove feature inapropriada
max_nan_percent = 0.1           # Para seleção de variáveis úteis
std_vars_min = 0.01             # Para seleção de variáveis úteis
clfs = {}                       # Dicionário para lista de classificadores a serem experimentados
disable_progressbar = True      # Para menos saídas no notebook

In [6]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    """Gerador de lista contendo número da classe e caminho do arquivo de acordo com a fonte da instância."""    
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [7]:
def load_instance(instance_path):
    """Função que carrega cada instância individualmente"""
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, sep=',', header=0)
        assert (df.columns == columns).all(), \
            f'Colunas inválidas no arquivo {str(instance_path)}: {str(df.columns.tolist())}'
        return df
    except Exception as e:
        raise Exception(f'Erro ao ler arquivo {instance_path}: {e}')

In [8]:
def extract_samples(df, class_code):
    # Obtém os rótulos das observações e seu conjunto inequívoco
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))       
    
    # Descarta os rótulos das observações e substitui todos os nan por 0
    # (requisito da biblioteca tsfresh)
    df_vars = df.drop('class', axis=1).fillna(0)  
    
    # Inicializa objetos que serão retornados
    df_samples_train = pd.DataFrame()
    df_samples_test = pd.DataFrame()
    y_train = []
    y_test = []
            
    # Descubre o número máximo de amostras em períodos normais, transitórios e em regime
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)

    # Define o número inicial de amostras para o período normal
    max_samples_normal = l_idx-f_idx+1-sample_size
    if (max_samples_normal) > 0:      
        num_normal_samples = min(max_samples_per_period, max_samples_normal)
        num_train_samples = int(split_range*num_normal_samples)
        num_test_samples = num_normal_samples - num_train_samples    
    else:
        num_train_samples = 0
        num_test_samples = 0
    
    # Define o número máximo de amostras por período transitório
    transient_code = class_code + 100    
    if transient_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição
        # no início do período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code)        
        max_transient_samples = l_idx-f_idx+1-sample_size
    else:
        max_transient_samples = 0            

    # Define o número máximo de amostras no período de regime
    if class_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou fim do período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1
        max_in_regime_samples = l_idx-f_idx+1-sample_size
    else:
        max_in_regime_samples = 0   
        
    # Descubre o número adequado de amostras em períodos normais, transitórios e em regime
    num_transient_samples = ceil(num_test_samples/2)
    num_in_regime_samples = num_test_samples - num_transient_samples
    if (max_transient_samples >= num_transient_samples) and \
       (max_in_regime_samples < num_in_regime_samples):
        num_in_regime_samples = max_in_regime_samples        
        num_transient_samples = min(num_test_samples-num_in_regime_samples, max_transient_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples >= num_in_regime_samples):
        num_transient_samples = max_transient_samples        
        num_in_regime_samples = min(num_test_samples-num_transient_samples, max_in_regime_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples < num_in_regime_samples):
        num_transient_samples = max_transient_samples
        num_in_regime_samples = max_in_regime_samples
        num_test_samples = num_transient_samples+num_in_regime_samples
    
    # Extrai amostras do período normal para treinamento e teste
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
    
    # Define a etapa correta e extrai amostras
    if (num_normal_samples) > 0:  
        if num_normal_samples == max_samples_normal:
            step_max = 1 
        else:
            step_max = (max_samples_normal-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Extrai amostras para treinamento
        sample_id = 0
        for idx in range(num_train_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_train = df_samples_train.append(df_sample)
            y_train.append(normal_class_code)
            sample_id += 1
    
        # Extrai amostras para teste
        sample_id = 0
        for idx in range(num_train_samples, num_train_samples+num_test_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(normal_class_code)
            sample_id += 1

    # Extrai amostras do período transitório (se existir) para teste
    if (num_transient_samples) > 0:    
        # Define a etapa correta e extrai amostras
        if num_transient_samples == max_transient_samples:
            step_max = 1 
        else:
            step_max = (max_transient_samples-1) // (max_samples_per_period-1)
        step_wanted = np.inf
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição no início deste período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code) 

        # Extrai amostras
        for idx in range(num_transient_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(transient_code)
            sample_id += 1
            
    # Extrai amostras do período em regime (se existir) para teste
    if (num_in_regime_samples) > 0:     
        # Define a etapa correta e extrai amostras
        if num_in_regime_samples == max_in_regime_samples:
            step_max = 1 
        else:
            step_max = (max_in_regime_samples-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou no final deste período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1

        # Extrai amostras
        for idx in range(num_in_regime_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(class_code)
            sample_id += 1

    return df_samples_train, y_train, df_samples_test, y_test              

In [9]:
def train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs):
    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)    
    for clf_name, clf in clfs.items():
        
        # Treino
        t0 = time()
        clf.fit(X_train)
        t_train = time() - t0

        # Teste
        t0 = time()
        y_pred = clf.predict(X_test)
        t_test = time() - t0

        # Calcula as metricas de desempenho
        ret = precision_recall_fscore_support(y_test, y_pred, average='micro')
        p, r, f1, _ = ret
        scores = scores.append({'CLASSIFICADOR': clf_name, 
                                'PRECISAO': p,
                                'REVOGACAO': r,
                                'F1': f1,
                                'TREINAMENTO [s]': t_train, 
                                'TESTE [s] ': t_test}, ignore_index=True)  
    return scores

In [10]:
# Gets all selected instances but maintains only those with any type of undesirable event
simulated_instances = pd.DataFrame(class_and_file_generator(data_path, 
                                                       real=False,
                                                       simulated=True, 
                                                       drawn=False),
                              columns=['class_code', 'instance_path'])
simulated_instances = simulated_instances.loc[simulated_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [11]:
# For each real instance with any type of undesirable event
scores = pd.DataFrame()
ignored_instances = 0
used_instances = 0

for i, row in simulated_instances.iterrows():
    # Loads the current instance
    class_code, instance_path = row
    print(f'Instância {i+1}: {instance_path}')
    df = load_instance(instance_path)
    
    # Ignores instances without sufficient normal periods
    normal_period_size = (df['class']==float(normal_class_code)).sum()
    if normal_period_size < min_normal_period_size:
        ignored_instances += 1
        print(f'\tignorado porque normal_period_size é insuficiente para treinamento ({normal_period_size})\n')
        continue
    used_instances += 1
        
    # Extracts samples from the current real instance
    ret = extract_samples(df, class_code)
    df_samples_train, y_train, df_samples_test, y_test = ret

    # Changes types of the labels (tsfresh's requirement)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # We want binary classification: 0 stands for inliers and 1 for outliers/anomalies
    # (pyod's requirement)
    y_train[y_train!=normal_class_code] = 1
    y_train[y_train==normal_class_code] = 0   
    y_test[y_test!=normal_class_code] = 1
    y_test[y_test==normal_class_code] = 0
    
    # Drops the bad vars
    good_vars = np.isnan(df_samples_train[vars]).mean(0) <= max_nan_percent
    std_vars = np.nanstd(df_samples_train[vars], 0)
    good_vars &= (std_vars > std_vars_min)    
    good_vars = list(good_vars.index[good_vars])
    bad_vars = list(set(vars)-set(good_vars))
    df_samples_train.drop(columns=bad_vars, inplace=True, errors='ignore')
    df_samples_test.drop(columns=bad_vars, inplace=True, errors='ignore')
    
    # Normalizes the samples (zero mean and unit variance)
    scaler = preprocessing.StandardScaler()
    df_samples_train[good_vars] = scaler.fit_transform(df_samples_train[good_vars]).astype('float32')
    df_samples_test[good_vars] = scaler.transform(df_samples_test[good_vars]).astype('float32')
    
    # Extracts features from samples
    X_train = extract_features(df_samples_train, 
                               column_id='id', 
                               column_sort='timestamp', 
                               default_fc_parameters=df_fc_p,
                               impute_function=impute,
                               n_jobs=0,
                               disable_progressbar=disable_progressbar)
    X_train = X_train.reset_index(drop=True)
    X_test = extract_features(df_samples_test, 
                              column_id='id', 
                              column_sort='timestamp',
                              default_fc_parameters=df_fc_p,
                              impute_function=impute,
                              n_jobs=0,
                              disable_progressbar=disable_progressbar)
    X_test = X_test.reset_index(drop=True)

    # LISTA DE CLASSIFICADORES A SEREM EXPERIMENTADOS
    
    # AUTOENCODER - Combinações
    grid_params = {
    'hidden_neurons': [[8, 4, 4, 8], [4, 2, 2, 4]],
    'epochs': [10],
    'dropout': [0, 0.2],
    'batch_size': [4, 8]
    }
    
    for params in ParameterGrid(grid_params):
        clfs[f'AutoEncoder {params}'] = AutoEncoder(hidden_neurons=params['hidden_neurons'], hidden_activation='relu', output_activation='sigmoid', 
                                       loss='mae', optimizer='adam', epochs=params['epochs'], batch_size=params['batch_size'], 
                                       dropout_rate=params['dropout'], l2_regularizer=0.1, validation_size=0.1, preprocessing=False, 
                                       verbose=0, random_state=random_state, contamination=0.1)
        
    # Trains, tests and calculates the scores    
    scores = train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs)

Instância 1: data\1\SIMULATED_00001.csv
Instância 2: data\1\SIMULATED_00002.csv
Instância 3: data\1\SIMULATED_00003.csv
Instância 4: data\1\SIMULATED_00004.csv
Instância 5: data\1\SIMULATED_00005.csv
Instância 6: data\1\SIMULATED_00006.csv
Instância 7: data\1\SIMULATED_00007.csv
Instância 8: data\1\SIMULATED_00008.csv
Instância 9: data\1\SIMULATED_00009.csv
Instância 10: data\1\SIMULATED_00010.csv
Instância 11: data\1\SIMULATED_00011.csv
Instância 12: data\1\SIMULATED_00012.csv
Instância 13: data\1\SIMULATED_00013.csv
Instância 14: data\1\SIMULATED_00014.csv
Instância 15: data\1\SIMULATED_00015.csv
Instância 16: data\1\SIMULATED_00016.csv
Instância 17: data\1\SIMULATED_00017.csv
Instância 18: data\1\SIMULATED_00018.csv
Instância 19: data\1\SIMULATED_00019.csv
Instância 20: data\1\SIMULATED_00020.csv
Instância 21: data\1\SIMULATED_00021.csv
Instância 22: data\1\SIMULATED_00022.csv
Instância 23: data\1\SIMULATED_00023.csv
Instância 24: data\1\SIMULATED_00024.csv
Instância 25: data\1\SIMU

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 156: data\5\SIMULATED_00026.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 157: data\5\SIMULATED_00027.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 158: data\5\SIMULATED_00028.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 159: data\5\SIMULATED_00029.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 160: data\5\SIMULATED_00030.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 161: data\5\SIMULATED_00031.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 162: data\5\SIMULATED_00032.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 163: data\5\SIMULATED_00033.csv
	ignorado porque normal_period_size é insuficiente para treiname

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 231: data\5\SIMULATED_00101.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 232: data\5\SIMULATED_00102.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 233: data\5\SIMULATED_00103.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 234: data\5\SIMULATED_00104.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 235: data\5\SIMULATED_00105.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 236: data\5\SIMULATED_00106.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 237: data\5\SIMULATED_00107.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 238: data\5\SIMULATED_00108.csv
	ignorado porque normal_period_size é insuficiente para treiname

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 304: data\5\SIMULATED_00174.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 305: data\5\SIMULATED_00175.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 306: data\5\SIMULATED_00176.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 307: data\5\SIMULATED_00177.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 308: data\5\SIMULATED_00178.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 309: data\5\SIMULATED_00179.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 310: data\5\SIMULATED_00180.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 311: data\5\SIMULATED_00181.csv
	ignorado porque normal_period_size é insuficiente para treiname

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 376: data\5\SIMULATED_00246.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 377: data\5\SIMULATED_00247.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 378: data\5\SIMULATED_00248.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 379: data\5\SIMULATED_00249.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 380: data\5\SIMULATED_00250.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 381: data\5\SIMULATED_00251.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 382: data\5\SIMULATED_00252.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 383: data\5\SIMULATED_00253.csv
	ignorado porque normal_period_size é insuficiente para treiname

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 450: data\5\SIMULATED_00320.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 451: data\5\SIMULATED_00321.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 452: data\5\SIMULATED_00322.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 453: data\5\SIMULATED_00323.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 454: data\5\SIMULATED_00324.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 455: data\5\SIMULATED_00325.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 456: data\5\SIMULATED_00326.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 457: data\5\SIMULATED_00327.csv
	ignorado porque normal_period_size é insuficiente para treiname

	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 523: data\5\SIMULATED_00393.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 524: data\5\SIMULATED_00394.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 525: data\5\SIMULATED_00395.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 526: data\5\SIMULATED_00396.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 527: data\5\SIMULATED_00397.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 528: data\5\SIMULATED_00398.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 529: data\5\SIMULATED_00399.csv
	ignorado porque normal_period_size é insuficiente para treinamento (500)

Instância 530: data\5\SIMULATED_00400.csv
	ignorado porque normal_period_size é insuficiente para treiname

Instância 633: data\6\SIMULATED_00064.csv
Instância 634: data\6\SIMULATED_00065.csv
Instância 635: data\6\SIMULATED_00066.csv
Instância 636: data\6\SIMULATED_00067.csv
Instância 637: data\6\SIMULATED_00068.csv
Instância 638: data\6\SIMULATED_00069.csv
Instância 639: data\6\SIMULATED_00070.csv
Instância 640: data\6\SIMULATED_00071.csv
Instância 641: data\6\SIMULATED_00072.csv
Instância 642: data\6\SIMULATED_00073.csv
Instância 643: data\6\SIMULATED_00074.csv
Instância 644: data\6\SIMULATED_00075.csv
Instância 645: data\6\SIMULATED_00076.csv
Instância 646: data\6\SIMULATED_00077.csv
Instância 647: data\6\SIMULATED_00078.csv
Instância 648: data\6\SIMULATED_00079.csv
Instância 649: data\6\SIMULATED_00080.csv
Instância 650: data\6\SIMULATED_00081.csv
Instância 651: data\6\SIMULATED_00082.csv
Instância 652: data\6\SIMULATED_00083.csv
Instância 653: data\6\SIMULATED_00084.csv
Instância 654: data\6\SIMULATED_00085.csv
Instância 655: data\6\SIMULATED_00086.csv
Instância 656: data\6\SIMULATED_00

Instância 829: data\8\SIMULATED_00045.csv
Instância 830: data\8\SIMULATED_00046.csv
Instância 831: data\8\SIMULATED_00047.csv
Instância 832: data\8\SIMULATED_00048.csv
Instância 833: data\8\SIMULATED_00049.csv
Instância 834: data\8\SIMULATED_00050.csv
Instância 835: data\8\SIMULATED_00051.csv
Instância 836: data\8\SIMULATED_00052.csv
Instância 837: data\8\SIMULATED_00053.csv
Instância 838: data\8\SIMULATED_00054.csv
Instância 839: data\8\SIMULATED_00055.csv
Instância 840: data\8\SIMULATED_00056.csv
Instância 841: data\8\SIMULATED_00057.csv
Instância 842: data\8\SIMULATED_00058.csv
Instância 843: data\8\SIMULATED_00059.csv
Instância 844: data\8\SIMULATED_00060.csv
Instância 845: data\8\SIMULATED_00061.csv
Instância 846: data\8\SIMULATED_00062.csv
Instância 847: data\8\SIMULATED_00063.csv
Instância 848: data\8\SIMULATED_00064.csv
Instância 849: data\8\SIMULATED_00065.csv
Instância 850: data\8\SIMULATED_00066.csv
Instância 851: data\8\SIMULATED_00067.csv
Instância 852: data\8\SIMULATED_00

## Resultados

Os resultados obtidos com os métodos implementados são apresentados abaixo.

In [12]:
print(f'Número de instâncias utilizadas: {used_instances}')
print(f'Número de instâncias ignoradas: {ignored_instances}')

Número de instâncias utilizadas: 426
Número de instâncias ignoradas: 439


In [13]:
print(f'Características utilizadas: {list(df_fc_p.keys())}')

Características utilizadas: ['median', 'mean', 'standard_deviation', 'variance', 'root_mean_square', 'maximum', 'minimum']


Os comandos a seguir permitem salvar e recuperar os resultados de/para um arquivo CSV de forma conveniente.

In [14]:
scores.to_csv(r'./results/1-1_anomaly_detection_scores_simuladas.csv')

### Métricas em formato tabular

As tabelas a seguir apresentam as médias e o desvio padrão das métricas, respectivamente. Ambos são ordenados pela medida-F1.

In [15]:
# Médias
mean_score_table = scores.groupby('CLASSIFICADOR').mean().sort_values(by=['F1'], ascending=False)
mean_score_table.to_csv(r'./results/1-1_anomaly_detection_scores_medias_simuladas.csv')
mean_score_table

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
"AutoEncoder {'batch_size': 8, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.724570,0.724570,0.724570,1.412069,0.031674
"AutoEncoder {'batch_size': 8, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.724178,0.724178,0.724178,1.454235,0.031684
"AutoEncoder {'batch_size': 8, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.723592,0.723592,0.723592,1.451255,0.032959
"AutoEncoder {'batch_size': 4, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.723396,0.723396,0.723396,1.442581,0.031729
"AutoEncoder {'batch_size': 8, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.723396,0.723396,0.723396,1.400502,0.032816
"AutoEncoder {'batch_size': 4, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.722418,0.722418,0.722418,1.439893,0.031735
"AutoEncoder {'batch_size': 4, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.722222,0.722222,0.722222,1.419095,0.034683
"AutoEncoder {'batch_size': 4, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.721440,0.721440,0.721440,1.379508,0.036941


In [16]:
# Desvios Padrão
std_score_table = scores.groupby('CLASSIFICADOR').std().sort_values(by=['F1'], ascending=True)
std_score_table.to_csv(r'./results/1-1_anomaly_detection_scores_desvios_padrao_simuladas.csv')
std_score_table

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
"AutoEncoder {'batch_size': 4, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.183019,0.183019,0.183019,0.389287,0.077983
"AutoEncoder {'batch_size': 8, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.183493,0.183493,0.183493,0.571998,0.017203
"AutoEncoder {'batch_size': 4, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.183571,0.183571,0.183571,0.776828,0.001317
"AutoEncoder {'batch_size': 4, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.183586,0.183586,0.183586,0.597415,0.042427
"AutoEncoder {'batch_size': 4, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.183849,0.183849,0.183849,0.693377,0.001208
"AutoEncoder {'batch_size': 8, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [8, 4, 4, 8]}",0.184099,0.184099,0.184099,0.832642,0.026224
"AutoEncoder {'batch_size': 8, 'dropout': 0.2, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.184138,0.184138,0.184138,0.825590,0.001258
"AutoEncoder {'batch_size': 8, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}",0.184370,0.184370,0.184370,0.695747,0.001310


In [17]:
# Calcular tempo total do notebook Jupyter
print(f'Tempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')

Tempo total de execução (hh:mm:ss.ms): 1:24:26.742708


## Análise

Tendo em vista os resultados apresentados acima, foram identificados os melhores parâmetros de camadas para o classificador de rede neural:

- Autoencoder: {'batch_size': 8, 'dropout': 0, 'epochs': 10, 'hidden_neurons': [4, 2, 2, 4]}	

Esses parâmetros serão utilizados na etapa subsequente (avaliação em instâncias reais).